# SEMA-1D

SEMA-1D is a fine-tuned ESM-1v model aimed to predict epitope resiudes based on antigen protein sequence

The MIT License (MIT)
Copyright (c) 2016 AYLIEN
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

## Requirements 

In [1]:
import os
# set cuda params
# 'TORCH_HOME'directory will be used to save origenal esm-1v weights
os.environ['TORCH_HOME'] = "../torch_hub"
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

In [2]:
import scipy
import sklearn
import math

import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset
from torch import nn

import transformers
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import Trainer, TrainingArguments, EvalPrediction

import esm
from esm.pretrained import load_model_and_alphabet_hub

from sklearn.metrics import r2_score, mean_squared_error

from tqdm import tqdm

2024-02-05 17:49:54.181357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 17:49:54.412502: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-05 17:49:55.866685: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-05 17:49:55.866819: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

## Set model

In [3]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2, pretrained_no = 1):
        super().__init__()
        self.num_labels = num_labels
        self.model_name = "esm1v_t33_650M_UR90S_" + str(pretrained_no) 
        
        self.esm1v, self.esm1v_alphabet = load_model_and_alphabet_hub(self.model_name)        
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)

In [4]:
class PDB_Dataset(Dataset):
    def __init__(self, df, label_type ='regression'):
        self.df = df
        _, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
        self.esm1v_batch_converter = esm1v_alphabet.get_batch_converter()
        self.label_type = label_type

    def __getitem__(self, idx):
        item = {}
        _, _, esm1b_batch_tokens = self.esm1v_batch_converter([('' , ''.join(self.df.iloc[idx,0])[:1022])])
        item['token_ids'] = esm1b_batch_tokens
        item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1][:1022]),0)

        return item

    def __len__(self):
        return len(self.df)
    
test_set = pd.read_csv('../data/sema_2.0/test_set.csv')
test_set = test_set.groupby('pdb_id_chain').agg({'resi_pos': list,
                                 'resi_aa': list,
                                 'contact_number_binary': list}).reset_index()
test_ds = PDB_Dataset(test_set[['resi_aa', 'contact_number_binary']],
                      label_type ='regression')

/ayb/vol2/home/dumerenkov/anaconda3/lib/python3.9/site-packages/esm/pretrained.py:215: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn(


In [5]:
model = ESM1vForTokenClassification(pretrained_no = 1).cuda()

## Prediction old train

In [6]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_{str(ens_idx)}.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs[:1022]]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:05<00:00, 19.15it/s]
/tmp/ipykernel_38737/2412427052.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


Model  0 0.6998000036773763
Model  1 0.7205089358946237
Model  2 0.7101373539055037
Model  3 0.7066858746776123
Model  4 0.7225313304669723

Ensemble  0.7484022335708338
Masked ensemble  0.7148341953344474


### Prediction new train

In [7]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_{str(ens_idx)}_new.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs[:1022]]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:05<00:00, 19.26it/s]
/tmp/ipykernel_38737/4106030711.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


Model  0 0.6705991501882089
Model  1 0.6868634516738169
Model  2 0.6902378668020495
Model  3 0.6815426502890445
Model  4 0.6941515113497089

Ensemble  0.7228392376447951
Masked ensemble  0.6908910785155113


## ESM2-3B

In [30]:
class PDB_Dataset(Dataset):
    def __init__(self, df, label_type ='regression'):
        self.df = df
        _, esm1v_alphabet = esm.pretrained.esm1v_t33_650M_UR90S_1()
        self.esm1v_batch_converter = esm1v_alphabet.get_batch_converter()
        self.label_type = label_type

    def __getitem__(self, idx):
        item = {}
        _, _, esm1b_batch_tokens = self.esm1v_batch_converter([('' , ''.join(self.df.iloc[idx,0]))])
        item['token_ids'] = esm1b_batch_tokens
        item['labels'] = torch.unsqueeze(torch.LongTensor(self.df.iloc[idx, 1]),0)

        return item

    def __len__(self):
        return len(self.df)

In [3]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2, pretrained_no = 1):
        super().__init__()
        self.num_labels = num_labels
        self.model_name = esm.pretrained.esm2_t36_3B_UR50D()  
        
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm2_t36_3B_UR50D()       
        self.classifier = nn.Linear(1280*2, self.num_labels)

    def forward(self, token_ids, labels = None):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[36])['representations'][36]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)
    
model = ESM1vForTokenClassification(pretrained_no = 1).cuda()

test_set = pd.read_csv('../data/sema_2.0/test_set.csv')
test_set = test_set.groupby('pdb_id_chain').agg({'resi_pos': list,
                                 'resi_aa': list,
                                 'contact_number_binary': list}).reset_index()
test_ds = PDB_Dataset(test_set[['resi_aa', 'contact_number_binary']],
                      label_type ='regression')

In [32]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_ESM2_{str(ens_idx)}_3B.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

for idx in range(5):
    test_set[f'ESM3B_new_{idx}'] = res[idx]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:17<00:00,  5.93it/s]


Model  0 0.7486206255697889
Model  1 0.7423144889945726
Model  2 0.7333851822156843
Model  3 0.7284082462914232
Model  4 0.7287659664608829

Ensemble  0.7657870297401864
Masked ensemble  0.7311150293594467


/tmp/ipykernel_38737/910925273.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


In [41]:
test_set['ESM3B_new_mean'] = test_set.apply(lambda row: np.mean(np.stack([row[f'ESM3B_new_{i}'] for i in range(5)], axis = 0), axis = 0), axis = 1)

In [46]:
test_set['contact_number_target'] = test_set['contact_number_binary'].apply(lambda x: [t>0 for t in x])

## Optimal threshold

In [50]:
import numpy as np
from sklearn.metrics import roc_curve, roc_auc_score
labels = [x for xs in test_set['contact_number_target'].tolist() for x in xs]
preds = [x for xs in test_set['ESM3B_new_mean'].tolist() for x in xs]
fpr, tpr, thresholds = roc_curve(labels, preds, pos_label=1)

optimal_threshold_ind = np.argmin(np.sqrt((fpr)**2 + (1-tpr)**2))
print(optimal_threshold_ind, thresholds[optimal_threshold_ind])
print(roc_auc_score(labels, preds))

2799 0.36172485
0.7657870297401864


In [80]:
fpr[len(thresholds[thresholds>0.361])]

0.34201036736533696

In [81]:
tpr[len(thresholds[thresholds>0.361])]

0.731418918918919

In [13]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_ESM2_{str(ens_idx)}_3B_old.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

for idx in range(5):
    test_set[f'ESM3B_old_{idx}'] = res[idx]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:17<00:00,  5.66it/s]
/tmp/ipykernel_38737/4221834530.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


Model  0 0.7383896558196128
Model  1 0.7451982678038079
Model  2 0.7490490062197086
Model  3 0.7476716542861319
Model  4 0.7407926752817726

Ensemble  0.7663257514614005
Masked ensemble  0.7234157139216113


## ESM2-650

In [15]:
class ESM1vForTokenClassification(nn.Module):

    def __init__(self, num_labels = 2, pretrained_no = 1):
        super().__init__()
        self.num_labels = num_labels
        self.model_name = esm.pretrained.esm2_t33_650M_UR50D()  
        
        self.esm1v, self.esm1v_alphabet = esm.pretrained.esm2_t33_650M_UR50D()#load_model_and_alphabet_hub(self.model_name)        
        self.classifier = nn.Linear(1280, self.num_labels)

    def forward(self, token_ids, labels = None):
                
        outputs = self.esm1v.forward(token_ids, repr_layers=[33])['representations'][33]
        outputs = outputs[:,1:-1,:]
        logits = self.classifier(outputs)

        return SequenceClassifierOutput(logits=logits)
    
model = ESM1vForTokenClassification(pretrained_no = 1).cuda()

## Prediction old train

In [17]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_ESM2_{str(ens_idx)}.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

for idx in range(5):
    test_set[f'ESM650_old_{idx}'] = res[idx]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:06<00:00, 16.00it/s]
/tmp/ipykernel_38737/2565486123.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


Model  0 0.7447024150798872
Model  1 0.7377723819153837
Model  2 0.7470255974788176
Model  3 0.745434289195382
Model  4 0.7543309822246181

Ensemble  0.7681633760408482
Masked ensemble  0.727870279194574


In [18]:
res=[]
for ens_idx in range(5):
    # path to model weights
    path = f'../models/sema_1d_ESM2_{str(ens_idx)}_new.pth'
    model.load_state_dict(torch.load(path))
    model.eval()
    model.cuda()

    with torch.no_grad():
        preds=[]
        for it in tqdm(test_ds):
            preds.append(model.forward(it['token_ids'].cuda())[0][0][:,1].cpu().numpy())
    res.append(preds)
    
    
for idx in range(5):
    y_true = [x for xs in test_set['contact_number_binary'].tolist() for x in xs]
    y_pred = [x for xs in [p for p in res[idx]] for x in xs]
    print('Model ', idx, sklearn.metrics.roc_auc_score([t>0 for t in y_true], y_pred))
    
print()
ens_pred = np.mean(np.array(res), axis = 0).tolist()
ens_pred = [x for xs in [p for p in ens_pred] for x in xs]

print('Ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in y_true], ens_pred))

masked_pred, masked_true = [],[]
for t, p in zip(y_true, ens_pred):
    if t!=-100:
        masked_pred.append(p)
        masked_true.append(t)
        
print('Masked ensemble ',sklearn.metrics.roc_auc_score([t>0 for t in masked_true], masked_pred))

for idx in range(5):
    test_set[f'ESM650_new_{idx}'] = res[idx]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [00:06<00:00, 16.01it/s]
/tmp/ipykernel_38737/543975243.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ens_pred = np.mean(np.array(res), axis = 0).tolist()


Model  0 0.7219064589437307
Model  1 0.7041977718717702
Model  2 0.7228035274117932
Model  3 0.726127742537162
Model  4 0.7297781665412877

Ensemble  0.7469601082446545
Masked ensemble  0.7148357616136957
